In [1]:
%matplotlib inline

from itertools import combinations
import json
import os
import sys

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from scipy.stats import spearmanr

from tqdm import tqdm

pd.set_option("display.max_columns", 101)
pd.set_option("display.float_format", lambda x: "%.2f" % x )

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src', 'data')
sys.path.append(src_dir)

# import my method from the source code
%aimport utils
%aimport download_planet

# load environment varibles
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

pass

/Users/bull/anaconda/envs/farmdrive/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Geographic Features

For our first pass at the data, our area of interest is just **Nakuru** county.

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('postgresql://localhost/farmdrive')
session = sessionmaker(bind=engine)()

In [29]:
county_name = 'Nakuru'
crop_table = 'maiz_p--ssa'
crop_name = 'maize'

In [48]:
# individual geojson polygons for each raster pixel
query = """
SELECT
json_build_object(
    'type', 'Feature',
    'id', (poly_pixels.x || '_' || poly_pixels.y),
    'geometry', ST_AsGeoJSON(1, poly_pixels.geom, 15, 2) :: JSON,
    'properties', json_build_object('{crop_name}_yield', poly_pixels.val)
)
FROM
  (SELECT (ST_PixelAsPolygons(ST_Union(ST_Clip("{crop_table}".rast, clipped_geom.geom)))).*
    FROM
      "{crop_table}",
      (SELECT county.geom FROM county WHERE county.county = '{county_name}') AS clipped_geom
    WHERE ST_Intersects("{crop_table}".rast, clipped_geom.geom)
  ) AS poly_pixels;
"""

query = query.format(crop_name=crop_name,
           crop_table=crop_table,
           county_name=county_name)

# Execute the query in the session
result = session.execute(query)

# for now, just get the first result. ultimately, we'll need them all.
aoi_raster = result.fetchall()
aoi_raster = aoi_raster[2]

geojson_from_postgis = aoi_raster['json_build_object']
geojson_from_postgis

{'geometry': {'coordinates': [[[35.9166783844273, 0.083326218967615],
    [36.0000117291493, 0.083326218967615],
    [36.0000117291493, -7.12575437e-06],
    [35.9166783844273, -7.12575437e-06],
    [35.9166783844273, 0.083326218967615]]],
  'crs': {'properties': {'name': 'EPSG:4326'}, 'type': 'name'},
  'type': 'Polygon'},
 'id': '8_3',
 'properties': {'maize_yield': 1},
 'type': 'Feature'}

In [56]:
geojson_from_postgis['geometry']['coordinates'][0] = geojson_from_postgis['geometry']['coordinates'][0][::-1]

In [57]:
from subprocess import check_output, CalledProcessError, STDOUT

planet_data = os.path.join(os.pardir,
                             'data',
                             'raw',
                             'planet')

county_data = os.path.join(planet_data, county_name)

county_pixel_dir = os.path.join(county_data,
                                geojson_from_postgis['id'])

asset_dir = os.path.join(county_data,
                         'assets')

os.makedirs(county_pixel_dir, exist_ok=True)

geojson_input = os.path.join(county_pixel_dir, 'geojson_epsg4326.geojson')
geojson_output = os.path.join(county_pixel_dir, 'geojson_epsg32637.geojson')

with open(geojson_input, 'w') as gj_file:
    json.dump(geojson_from_postgis, gj_file)

try:
    check_output(['ogr2ogr',
                  '-f',
                  'GeoJSON',
                  geojson_output,
                  '-t_srs',
                  'EPSG:32637',
                  geojson_input], stderr=STDOUT)

except CalledProcessError as e:
    print(e.output)

# Download Data

In [50]:
geo_json_geometry = geojson_from_postgis['geometry']

# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

# MAIZE harvest season in Kenya is Aug - Oct
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-07-31T00:00:00.000Z",
    "lte": "2016-10-31T00:00:00.000Z"
  }
}

# filter any images which are more than 10% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.05
  }
}

# create a filter that combines our geo and date filters
maize_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [51]:
def has_local_scene(scene_id, asset_type):
    glob_path = os.path.join(asset_dir, '{}_{}.tif'.format(sid, asset_type))
    scene_paths = glob(glob_path)    
    return len(scene_paths) > 0

[]

In [52]:
import time

# Product type, PSOrthoTile or REOrthoTile
search_type = 'PSOrthoTile'

# visual or analytic - which asset type to download
asset_type = 'analytic'


# get the planet scenes for our query
scene_ids = download_planet.run_search({'item_types': [search_type],
                                        'filter': maize_filter})

# check for scenes that we _don't_ already have
not_local_scene_ids = []
for sid in scene_ids:
    if not has_local_scene(sid, asset_type):
        not_local_scene_ids.append(sid)
    
# activate all of the planet scenes that we may want
res2 = download_planet.process_activation(download_planet.activate,
                                          not_local_scene_ids,
                                          search_type,
                                          asset_type)

for i in tqdm(range(100)):
    resie = download_planet.process_activation(download_planet.check_activation,
                                               not_local_scene_ids,
                                               search_type,
                                               asset_type)
    
    if all(resie):
        print('Activated!')
        break
    else:
        time.sleep(15)

results = download_planet.process_download(asset_dir,
                                           not_local_scene_ids,
                                           search_type,
                                           asset_type,
                                           True)

results
assert all(results)

Running query
273250_3739001_2016-10-21_0c76 {'analytic_xml': {'_permissions': ['download'], 'type': 'analytic_xml', 'status': 'inactive', '_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjI3MzI1MF8zNzM5MDAxXzIwMTYtMTAtMjFfMGM3NiIsICJjIjogIlBTT3J0aG9UaWxlIiwgInQiOiAiYW5hbHl0aWNfeG1sIiwgImN0IjogIml0ZW0tdHlwZSJ9', 'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjI3MzI1MF8zNzM5MDAxXzIwMTYtMTAtMjFfMGM3NiIsICJjIjogIlBTT3J0aG9UaWxlIiwgInQiOiAiYW5hbHl0aWNfeG1sIiwgImN0IjogIml0ZW0tdHlwZSJ9/activate', 'type': 'https://api.planet.com/data/v1/asset-types/analytic_xml'}}, 'analytic': {'_permissions': ['download'], 'type': 'analytic', 'status': 'inactive', '_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjI3MzI1MF8zNzM5MDAxXzIwMTYtMTAtMjFfMGM3NiIsICJjIjogIlBTT3J0aG9UaWxlIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0', 'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjI3MzI1MF8zNzM5MDAxXzIwMTYtMTAtMjFfMGM3NiIsICJjIjogIlBTT3J0aG9UaWxlIi

  0%|          | 0/100 [00:00<?, ?it/s]

Activation process started successfully
273250_3739001_2016-10-21_0c76: activating
Item not yet active: 273250_3739001_2016-10-21_0c76
273250_3639028_2016-10-21_0c76: activating
Item not yet active: 273250_3639028_2016-10-21_0c76
273250_3639128_2016-10-21_0c76: activating
Item not yet active: 273250_3639128_2016-10-21_0c76
272572_3739001_2016-10-20_0e16: activating
Item not yet active: 272572_3739001_2016-10-20_0e16
272572_3739101_2016-10-20_0e16: activating
Item not yet active: 272572_3739101_2016-10-20_0e16
272572_3639028_2016-10-20_0e16: activating
Item not yet active: 272572_3639028_2016-10-20_0e16
272572_3639128_2016-10-20_0e16: activating
Item not yet active: 272572_3639128_2016-10-20_0e16
270187_3739001_2016-10-17_0e19: activating
Item not yet active: 270187_3739001_2016-10-17_0e19
270187_3739101_2016-10-17_0e19: activating
Item not yet active: 270187_3739101_2016-10-17_0e19
270187_3639128_2016-10-17_0e19: activating
Item not yet active: 270187_3639128_2016-10-17_0e19
270187_363

  1%|          | 1/100 [00:35<58:59, 35.75s/it]

273250_3739001_2016-10-21_0c76: activating
Item not yet active: 273250_3739001_2016-10-21_0c76
273250_3639028_2016-10-21_0c76: activating
Item not yet active: 273250_3639028_2016-10-21_0c76
273250_3639128_2016-10-21_0c76: activating
Item not yet active: 273250_3639128_2016-10-21_0c76
272572_3739001_2016-10-20_0e16: activating
Item not yet active: 272572_3739001_2016-10-20_0e16
272572_3739101_2016-10-20_0e16: activating
Item not yet active: 272572_3739101_2016-10-20_0e16
272572_3639028_2016-10-20_0e16: activating
Item not yet active: 272572_3639028_2016-10-20_0e16
272572_3639128_2016-10-20_0e16: activating
Item not yet active: 272572_3639128_2016-10-20_0e16
270187_3739001_2016-10-17_0e19: activating
Item not yet active: 270187_3739001_2016-10-17_0e19
270187_3739101_2016-10-17_0e19: activating
Item not yet active: 270187_3739101_2016-10-17_0e19
270187_3639128_2016-10-17_0e19: activating
Item not yet active: 270187_3639128_2016-10-17_0e19
270187_3639028_2016-10-17_0e19: activating
Item no

...

273250_3739001_2016-10-21_0c76: active
273250_3639028_2016-10-21_0c76: active
273250_3639128_2016-10-21_0c76: active
272572_3739001_2016-10-20_0e16: active
272572_3739101_2016-10-20_0e16: active
272572_3639028_2016-10-20_0e16: active
272572_3639128_2016-10-20_0e16: active
270187_3739001_2016-10-17_0e19: active
270187_3739101_2016-10-17_0e19: active
270187_3639128_2016-10-17_0e19: active
270187_3639028_2016-10-17_0e19: active
266021_3639028_2016-10-12_0e30: active
266021_3739101_2016-10-12_0e30: active
266021_3639128_2016-10-12_0e30: active
221633_3639128_2016-08-15_0e30: active
221633_3639028_2016-08-15_0e30: active
221633_3739001_2016-08-15_0e30: active
262306_3639128_2016-10-07_0e0e: active
262306_3639028_2016-10-07_0e0e: active
262306_3739001_2016-10-07_0e0e: active
239495_3739101_2016-09-11_0c78: active
239495_3739001_2016-09-11_0c78: active
239495_3639128_2016-09-11_0c78: active
239495_3639028_2016-09-11_0c78: active
256802_3739101_2016-10-01_0e14: active
256802_3639128_2016-10-01

 17%|█▋        | 17/100 [10:59<54:03, 39.08s/it]

273250_3739001_2016-10-21_0c76: active
273250_3639028_2016-10-21_0c76: active
273250_3639128_2016-10-21_0c76: active
272572_3739001_2016-10-20_0e16: active
272572_3739101_2016-10-20_0e16: active
272572_3639028_2016-10-20_0e16: active
272572_3639128_2016-10-20_0e16: active
270187_3739001_2016-10-17_0e19: active
270187_3739101_2016-10-17_0e19: active
270187_3639128_2016-10-17_0e19: active
270187_3639028_2016-10-17_0e19: active
266021_3639028_2016-10-12_0e30: active
266021_3739101_2016-10-12_0e30: active
266021_3639128_2016-10-12_0e30: active
221633_3639128_2016-08-15_0e30: active
221633_3639028_2016-08-15_0e30: active
221633_3739001_2016-08-15_0e30: active
262306_3639128_2016-10-07_0e0e: active
262306_3639028_2016-10-07_0e0e: active
262306_3739001_2016-10-07_0e0e: active
239495_3739101_2016-09-11_0c78: active
239495_3739001_2016-09-11_0c78: active
239495_3639128_2016-09-11_0c78: active
239495_3639028_2016-09-11_0c78: active
256802_3739101_2016-10-01_0e14: active
256802_3639128_2016-10-01

 18%|█▊        | 18/100 [11:36<52:40, 38.54s/it]

273250_3739001_2016-10-21_0c76: active
273250_3639028_2016-10-21_0c76: active
273250_3639128_2016-10-21_0c76: active
272572_3739001_2016-10-20_0e16: active
272572_3739101_2016-10-20_0e16: active
272572_3639028_2016-10-20_0e16: active
272572_3639128_2016-10-20_0e16: active
270187_3739001_2016-10-17_0e19: active
270187_3739101_2016-10-17_0e19: active
270187_3639128_2016-10-17_0e19: active
270187_3639028_2016-10-17_0e19: active
266021_3639028_2016-10-12_0e30: active
266021_3739101_2016-10-12_0e30: active
266021_3639128_2016-10-12_0e30: active
221633_3639128_2016-08-15_0e30: active
221633_3639028_2016-08-15_0e30: active
221633_3739001_2016-08-15_0e30: active
262306_3639128_2016-10-07_0e0e: active
262306_3639028_2016-10-07_0e0e: active
262306_3739001_2016-10-07_0e0e: active
239495_3739101_2016-09-11_0c78: active
239495_3739001_2016-09-11_0c78: active
239495_3639128_2016-09-11_0c78: active
239495_3639028_2016-09-11_0c78: active
256802_3739101_2016-10-01_0e14: active
256802_3639128_2016-10-01

# Merge tiles for each pixel and crop to json

In [71]:
from glob import glob

import rasterio
from rasterio import features
from rasterio.merge import merge

paths = [glob(county_data + os.sep + 'assets' + os.sep + '{}_{}.tif'.format(sid, asset_type))[0] \
    for sid in scene_ids]

planet_crs = {'proj': 'utm', 'zone': 37, 'ellps': 'WGS84', 'datum': 'WGS84', 'units':'m', 'no_defs': True}
srcs = [rasterio.open(p, crs=planet_crs) for p in paths]

with open(geojson_output) as gj_file:
    reprojgeoj = json.load(gj_file)

out_image, out_transform = my_merge(srcs, bounds=features.bounds(reprojgeoj))

# save the resulting raster
out_meta = srcs[0].meta.copy()
out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform})

raster_merged_path = os.path.join(county_pixel_dir, geojson_from_postgis['id'] + '.tif')

with rasterio.open(raster_merged_path, "w", **out_meta) as dest:
    dest.write(out_image)

ValueError: invalid window: row range (3197208, 8000)

In [72]:
%debug

> /Users/bull/anaconda/envs/farmdrive/lib/python3.5/site-packages/rasterio/windows.py(297)evaluate()
    295     if not r_stop >= r_start:
    296         raise ValueError(
--> 297             "invalid window: row range (%d, %d)" % (r_start, r_stop))
    298     c_start = c[0] or 0
    299     if c_start < 0:

ipdb> u
> <ipython-input-70-19836e8220c4>(104)my_merge()
    102                         window=src_window,
    103                         boundless=False,
--> 104                         masked=True)
    105 
    106         # 5. Copy elements of temp into dest.

ipdb> src_window
Window(col_off=0, row_off=3197208, num_cols=808, num_rows=-3189208)
ipdb> src.bounds
BoundingBox(left=163500.0, bottom=9975500.0, right=188500.0, top=10000500.0)
ipdb> dst.bounds
*** NameError: name 'dst' is not defined
ipdb> dst_window
Window(col_off=2164, row_off=0, num_cols=808, num_rows=-3189208)
ipdb> dst_window.__dict__
*** AttributeError: 'Window' object has no attribute '__dict__'
ipdb> dst_win

# Crop to pixels

Now done using bounds above.

In [ ]:
# import gdal

# # Crop back to AOI
# tif_loc = '../data/raw/planet/nakuru/11_1/271486_3739102_2016-10-19_0c65_analytic.tif'
# output_file = os.path.splitext(tif_loc)[0] + '_clipped.tif'

# import json
# with open('../test.geojson', 'w') as tg:
#     json.dump(aoi_raster['json_build_object'], tg)

# # GDAL Warp crops the image by our AOI, and saves it
# gdal.Warp(output_file,
#           tif_loc,
#           srcSRS = 'EPSG:32637',
# #           dstSRS = 'WGS64',
#           format='GTiff',
#           dstAlpha=True,
#           cutlineDSName='../test2.geojson',
#           cropToCutline=True)

In [ ]:
# import rasterio
# from rasterio.mask import mask

# # Crop back to AOI
# tif_loc = '../data/raw/planet/nakuru/11_1/11_rio.tif'
# output_file = os.path.splitext(tif_loc)[0] + '_clipped.tif'

# with open('../test2.geojson') as tg:
#     reprojgeoj = json.load(tg)

# planet_crs = {'proj': 'utm', 'zone': 37, 'ellps': 'WGS84', 'datum': 'WGS84', 'units':'m', 'no_defs': True}
# # load the raster, mask it by the polygon and crop it
# with rasterio.open(tif_loc, crs=planet_crs) as src:
#     out_image, out_transform = mask(src,
#                                     [reprojgeoj],
#                                     crop=True)
    
#     #plt.imshow(out_image)
#     out_meta = src.meta.copy()

# # save the resulting raster  
# out_meta.update({"driver": "GTiff",
#     "height": out_image.shape[1],
#     "width": out_image.shape[2],
#     "transform": out_transform})

# with rasterio.open(output_file, "w", **out_meta) as dest:
#     dest.write(out_image)

In [84]:
from itertools import compress

test = ['a', 'b', 'c']
mask = [True, False, True]

list(compress(test, mask))

['a', 'c']